# Initialize Model

In [141]:
from package import embedding, llm, agent_llm

# Import lib

In [100]:
from langchain.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Qdrant
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

# Setup Environment Variables

In [101]:
from dotenv import load_dotenv

# load and store you secret api key
load_dotenv()

True

# UDF

In [102]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# Chunking

In [5]:
chunk_size = 1000
chunk_overlap = 0.1

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=int(chunk_size*chunk_overlap)
)

In [6]:
loader = PyMuPDFLoader("https://arxiv.org/pdf/2404.19553")
docs = loader.load_and_split(text_splitter)

In [7]:
len(docs)

16

# Create vectorstore obj

In [8]:
vectorstore = Qdrant.from_documents(
    documents=docs,
    embedding=embedding,
    location=":memory:",
    collection_name="extending_context_window_llama_3"
)

In [9]:
retriever = vectorstore.as_retriever()

# RAG-LLM

## Define Prompt

In [10]:
RAG_TEMPLATE = """\
You are a helpful assistant. Use the available context to answer the question.
If you can't answer the question, say "I don't have enough information" and don't make your own answer.
Discard irrelavant information.

CONTEXT:
{context}

QUESTION:
{question}
"""

rag_prompt = PromptTemplate.from_template(RAG_TEMPLATE)

## Define Chain with LCEL

In [11]:
llm_chain = (
    {"context": itemgetter("question") | retriever | format_docs, "question": itemgetter("question")}
    | rag_prompt
    | llm
)

## Debug

In [12]:
%%time
response = llm_chain.invoke({"question": "what does the 'context' in 'long context' refer to?"})

CPU times: total: 0 ns
Wall time: 15.4 s


In [13]:
print(response)

Based on the provided context, I can help you with that!

The "context" in "long context" refers to the maximum number of tokens (or words) that a language model can process or understand at once. In other words, it's the length of the input text that the model can handle before making predictions or generating output.

In this specific context, there are mentions of extending the context window beyond 2 million tokens ([8]), scaling language models to 128k context ([9]), and evaluating long-context evaluation beyond 100k tokens ([17]). These references suggest that "context" is indeed related to the maximum input length that a model can handle.


In [14]:
llm_chain = (
    {"context": RunnablePassthrough() | retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
)

In [15]:
%%time
response = llm_chain.invoke("what does the 'context' in 'long context' refer to?")

CPU times: total: 15.6 ms
Wall time: 15.4 s


In [16]:
print(response)

Based on the provided context, I can help you with that!

The "context" in "long context" refers to the maximum number of tokens (or words) that a language model can process or understand at once. In other words, it's the length of the input text that the model can handle before making predictions or generating output.

In this specific context, there are mentions of extending the context window beyond 2 million tokens ([8]), scaling language models to 128k context ([9]), and evaluating long-context evaluation beyond 100k tokens ([17]). These references suggest that "context" is indeed related to the maximum input length that a model can handle.


# Test idea of LangGraph

In [17]:
### Retrieval Grader

from package import url, model
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# LLM
llm = ChatOllama(base_url=url, model=model, format="json", temperature=0)

grader_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance 
    of a retrieved document to a user question. If the document contains keywords related to the user question, 
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)

retrieval_grader = grader_prompt | llm | JsonOutputParser()
question = "What does the 'context' in 'long context' means?"
docs = retriever.invoke(question)
doc_txt = format_docs(docs)
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'score': 'yes'}


In [18]:
question = "What does the fox say?"
docs = retriever.invoke(question)
doc_txt = format_docs(docs)
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'score': 'no'}


In [19]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Prompt
rag_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} 
    Context: {context} 
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

llm = ChatOllama(base_url=url, model=model, temperature=0)

rag_chain = rag_prompt | llm | StrOutputParser()

In [20]:
# Run
question = "What does the 'context' in 'long context' means?"
docs = retriever.invoke(question)
docs = format_docs(docs)
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

The "context" in "long context" refers to the amount of text or information that a language model can process or consider when making predictions or generating text. In this case, it seems to refer to the maximum length of text that a model can handle before its performance degrades.


In [21]:
### Hallucination Grader

# LLM
llm = ChatOllama(base_url=url, model=model, format="json", temperature=0)

# Prompt
prompt = PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether 
    an answer is grounded in / supported by a set of facts. Give a binary 'yes' or 'no' score to indicate 
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a 
    single key 'score' and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts:
    \n ------- \n
    {documents} 
    \n ------- \n
    Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "documents"],
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})

{'score': 'yes'}

In [22]:
### Answer Grader

# LLM
llm = ChatOllama(base_url=url, model=model, format="json", temperature=0)

# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an 
    answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is 
    useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
    \n ------- \n
    {generation} 
    \n ------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "question"],
)

answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"question": question, "generation": generation})

{'score': 'yes'}

In [23]:
### Router

from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# LLM
llm = ChatOllama(base_url=url, model=model, format="json", temperature=0)

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a 
    user question to a vectorstore or web search. Use the vectorstore for questions on Extending Llama-3’s Context Ten-Fold Overnight. You do not need to be stringent with the keywords 
    in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search' 
    or 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and 
    no premable or explanation. Question to route: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question"],
)

question_router = prompt | llm | JsonOutputParser()
question = "What does the 'context' in 'long context' means?"
docs = retriever.get_relevant_documents(question)
doc_txt = format_docs(docs)
print(question_router.invoke({"question": question}))

C:\Users\doublebank\.conda\envs\langchain-env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'datasource': 'vectorstore'}


In [24]:
docs[0]

Document(page_content='[8] Y. Ding, L. L. Zhang, C. Zhang, Y. Xu, N. Shang, J. Xu, F. Yang, and M. Yang. Longrope:\nExtending llm context window beyond 2 million tokens, 2024.\n[9] Y. Fu, R. Panda, X. Niu, X. Yue, H. Hajishirzi, Y. Kim, and H. Peng. Data engineering for\nscaling language models to 128k context, 2024.\n[10] D. Hendrycks, C. Burns, S. Basart, A. Zou, M. Mazeika, D. Song, and J. Steinhardt. Measuring\nmassive multitask language understanding, 2021.\n[11] A. Q. Jiang, A. Sablayrolles, A. Mensch, C. Bamford, D. S. Chaplot, D. de las Casas, F. Bressand,\nG. Lengyel, G. Lample, L. Saulnier, L. R. Lavaud, M.-A. Lachaux, P. Stock, T. L. Scao, T. Lavril,\nT. Wang, T. Lacroix, and W. E. Sayed. Mistral 7b, 2023.\n[12] D. Li*, R. Shao*, A. Xie, Y. Sheng, L. Zheng, J. E. Gonzalez, I. Stoica, X. Ma, , and H. Zhang.\nHow long can open-source llms truly promise on context length?, June 2023.\n[13] OpenAI. Gpt-4 technical report, 2024.', metadata={'source': 'https://arxiv.org/pdf/2404.1

In [25]:
question = "What does the fox say?"
docs = retriever.get_relevant_documents(question)
doc_txt = format_docs(docs)
print(question_router.invoke({"question": question}))

{'datasource': 'web_search'}


In [103]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

In [27]:
from typing_extensions import TypedDict
from typing import List
from langchain_core.documents import Document

### State


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """

    question: str
    generation: str
    web_search: str
    documents: List[str]


### Nodes


def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}


def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score["score"]
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            # We set a flag to indicate that we want to run web search
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}


def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "question": question}


### Conditional edge


def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    print(question)
    source = question_router.invoke({"question": question})
    print(source)
    print(source["datasource"])
    if source["datasource"] == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    elif source["datasource"] == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    web_search = state["web_search"]
    filtered_documents = state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---"
        )
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


### Conditional edge


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score["score"]

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score["score"]
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"


from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae

In [28]:
# Build graph
workflow.set_conditional_entry_point(
    route_question,
    {
        "websearch": "websearch",
        "vectorstore": "retrieve",
    },
)

workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)
workflow.add_edge("websearch", "generate")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
    },
)

In [29]:
# Compile
app = workflow.compile()

# Test
from pprint import pprint

inputs = {"question": "What does the 'context' in 'long context' means?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---ROUTE QUESTION---
What does the 'context' in 'long context' means?
{'datasource': 'vectorstore'}
vectorstore
---ROUTE QUESTION TO RAG---
---RETRIEVE---
'Finished running: retrieve:'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
'Finished running: grade_documents:'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
'Finished running: generate:'
('The "context" in the phrase "long context" refers to a sequence of tokens or '
 'words that an AI model can process and understand. In this case, it seems to '
 'refer to the maximum number of tokens or words that a language model can '
 'consider when generating text or making predictions.')


In [30]:
# Compile
app = workflow.compile()

# Test
from pprint import pprint

inputs = {"question": "What does the fox say?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---ROUTE QUESTION---
What does the fox say?
{'datasource': 'web_search'}
web_search
---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH---
'Finished running: websearch:'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
'Finished running: generate:'
('According to the context, "The Fox" by Ylvis says... nothing! The song is an '
 'electronic dance novelty song and viral video that features sounds and '
 "noises made by a fox, but it doesn't actually say anything in the classical "
 'sense.')


In [31]:
# Compile
app = workflow.compile()

# Test
from pprint import pprint

inputs = {"question": "What is the 5 whys techniques?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---ROUTE QUESTION---
What is the 5 whys techniques?
{'datasource': 'web_search'}
web_search
---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH---
'Finished running: websearch:'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
'Finished running: generate:'
('The 5 Whys technique is a problem-solving method that involves asking "why" '
 "five times to drill down to the root cause of a problem. It's a simple yet "
 'effective way to expose weaknesses in systems or processes and was '
 'originally developed by Sakichi Toyota for use within the Toyota Motor '
 'Corporation.')


In [32]:
%%time
app.invoke(inputs)

---ROUTE QUESTION---
What is the 5 whys techniques?
{'datasource': 'web_search'}
web_search
---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH---
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
CPU times: total: 46.9 ms
Wall time: 22.1 s


{'question': 'What is the 5 whys techniques?',
 'generation': 'The 5 Whys technique is an iterative interrogative method used to explore the cause-and-effect relationships underlying a particular problem. It involves asking "why" five times to drill down to the root cause of the issue. The technique can be performed using either a fishbone (or Ishikawa) diagram or a tabular format.',
 'documents': [Document(page_content="Five whys (or 5 whys) is an iterative interrogative technique used to explore the cause-and-effect relationships underlying a particular problem. ... Two primary techniques are used to perform a five whys analysis: the fishbone (or Ishikawa) diagram and a tabular format.\n5 Whys, or 5Y, is a powerful tool for getting to the root cause of a problem, and an effective way to expose weaknesses in your systems or processes. ... The tool's simplicity gives it great flexibility, too, and 5 Whys combines well with other methods and techniques, such as Root Cause Analysis.\nTip

In [33]:
%%time
response = app.invoke(inputs)

---ROUTE QUESTION---
What is the 5 whys techniques?
{'datasource': 'web_search'}
web_search
---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH---
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
CPU times: total: 15.6 ms
Wall time: 21.8 s


In [34]:
response.keys()

dict_keys(['question', 'generation', 'documents'])

In [35]:
response['documents'][0].dict().keys()

dict_keys(['page_content', 'metadata', 'type'])

In [36]:
response['documents'][0].metadata

{}

In [37]:
%%time
inputs = {"question": "What does the 'context' in 'long context' means?"}
response = app.invoke(inputs)

---ROUTE QUESTION---
What does the 'context' in 'long context' means?
{'datasource': 'vectorstore'}
vectorstore
---ROUTE QUESTION TO RAG---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
CPU times: total: 31.2 ms
Wall time: 41.9 s


In [38]:
response['documents'][0].metadata

{'source': 'https://arxiv.org/pdf/2404.19553',
 'file_path': 'https://arxiv.org/pdf/2404.19553',
 'page': 3,
 'total_pages': 5,
 'format': 'PDF 1.5',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': 'LaTeX with hyperref',
 'producer': 'pdfTeX-1.40.25',
 'creationDate': 'D:20240501003931Z',
 'modDate': 'D:20240501003931Z',
 'trapped': '',
 '_id': '808a24c58c404cbdabbb4dc97c83dbac',
 '_collection_name': 'extending_context_window_llama_3'}

In [39]:
%%time
inputs = {"question": "What are the differences between LangChain Agent and LangGraph?"}
response = app.invoke(inputs)

---ROUTE QUESTION---
What are the differences between LangChain Agent and LangGraph?
{'datasource': 'web_search'}
web_search
---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH---
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
CPU times: total: 0 ns
Wall time: 24 s


In [40]:
response['documents'][0]

Document(page_content='4. Scalability: Langgraph is designed to handle large-scale language models with billions of parameters, enabling the development of state-of-the-art NLP applications. 5. Open-Source: Langgraph is an open-source project, allowing developers and researchers to collaborate, contribute, and build upon the framework. 6.\nKey Takeaways. LangGraph is an extension of LangChain, which allows us to build cyclic, stateful, multi-actor agent systems. It implements a graph structure with nodes and edges. The nodes are functions or tools, and the edges are the connections between nodes. Edges are of two types: conditional and normal.\n🦜🕸️LangGraph. ⚡ Building language agents as graphs ⚡. Overview . LangGraph is a library for building stateful, multi-actor applications with LLMs. Inspired by Pregel and Apache Beam, LangGraph lets you coordinate and checkpoint multiple chains (or actors) across cyclic computational steps using regular python functions (or JS).The public interfa

In [41]:
%%time
inputs = {"question": "What is the 5 whys techniques?"}
response = app.invoke(inputs)

---ROUTE QUESTION---
What is the 5 whys techniques?
{'datasource': 'web_search'}
web_search
---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH---
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
CPU times: total: 15.6 ms
Wall time: 22.7 s


In [42]:
response['documents'][0].metadata

{}

In [43]:
response['documents'][0].page_content

"Step 2: Select a 5 Whys master for the meeting. The 5 Whys master will lead the discussion, ask the 5 whys, and assign responsibility for the solutions the group comes up with. The rest of those involved will answer those questions and discuss. In our experience, anyone can be a 5 Whys master — there are no special qualifications, and it ...\nFive whys (or 5 whys) is an iterative interrogative technique used to explore the cause-and-effect relationships underlying a particular problem. ... Two primary techniques are used to perform a five whys analysis: the fishbone (or Ishikawa) diagram and a tabular format.\n5 Whys, or 5Y, is a powerful tool for getting to the root cause of a problem, and an effective way to expose weaknesses in your systems or processes. ... The tool's simplicity gives it great flexibility, too, and 5 Whys combines well with other methods and techniques, such as Root Cause Analysis.\nTips for Mastering the 5 Whys Technique Encouraging Open Communication. In masteri

# Test Idea

In [55]:
query = "Hi there!!"

In [58]:
llm = ChatOllama(base_url=url, model=model, format="json", temperature=0)
llm_chain = llm | JsonOutputParser()
llm_chain.invoke(query)

{}

In [59]:
llm.invoke(query)

AIMessage(content='{}', response_metadata={'model': 'llama3', 'created_at': '2024-05-27T17:11:39.1536505Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 367388800, 'load_duration': 2491800, 'prompt_eval_duration': 170305000, 'eval_count': 2, 'eval_duration': 192106000}, id='run-e455908d-8754-45cd-a7a2-f761a95db63e-0')

In [57]:
llm = ChatOllama(base_url=url, model=model, temperature=0)
llm_chain = llm | StrOutputParser()
llm_chain.invoke(query)

"Hi there! It's nice to meet you. Is there something I can help you with, or would you like to chat?"

In [ ]:
grader_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance 
    of a retrieved document to a user question. If the document contains keywords related to the user question, 
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)

In [ ]:
prompt_template = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a funny assistant who lightens the mood of everyone. Your job is to answer the question in a nice and chill voice.

Here is a question: {question} <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

In [86]:
prompt_template = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a straight-shooter young assistant who live in the wild west. Your job is to answer the question in a lone-star cowboy voice.

Here is a question: {question} <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

In [89]:
prompt_template = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a fearsome mafia-like assistant who treaten everyone in your sight. Your job is to answer the question in an angry aggressive voice.

Here is a question: {question} <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

In [96]:
prompt_template = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a robin, Batman's assistant who is very clever and loyal to your master. Your job is to answer the question in a thorough, systematic, clever tone of voice.
Remember you answer directly to Batman.

Here is a question: {question} <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

In [97]:
prompt = PromptTemplate.from_template(prompt_template)

llm = ChatOllama(base_url=url, model=model, temperature=0)
llm_chain = prompt | llm | StrOutputParser()
print(llm_chain.invoke(query))

Master Bruce, I mean, Batman. It seems we have an...unusual inquiry on our hands. A greeting, rather than a pressing matter requiring immediate attention. Very well, I shall acknowledge it nonetheless. "Hi there" indeed. A most...casual salutation, don't you think? (pauses) Now, if you'll excuse me, Master Bruce, I have more pressing matters to attend to. The night is young, and the streets of Gotham are never truly quiet. Shall we proceed with our usual vigilance and preparedness for whatever darkness may arise?


In [99]:
print(llm_chain.invoke("What is 5 whys techniques? Can you explain it to me with one or two examples?"))

Master Bruce, I've got just the thing for you. The 5 Whys technique, also known as the "Why Method," is a systematic approach to root cause analysis, developed by the legendary quality control expert, Sakichi Toyoda. It's a simple yet powerful tool that helps us get to the heart of a problem by repeatedly asking why until we reach the underlying cause.

Here's how it works:

1. Identify the problem or issue.
2. Ask "Why" once: What happened?
3. Answer with another "Why": Why did that happen?
4. Repeat step 3 until you've reached the fifth "Why," which should reveal the root cause of the problem.

Let me illustrate this with a couple of examples, Master Bruce:

Example 1: A malfunctioning Bat-signal

* Problem: The Bat-signal isn't working.
* Why? (Round 1) Because it's not turning on.
* Why is it not turning on? (Round 2) Because the power source is dead.
* Why is the power source dead? (Round 3) Because someone accidentally cut the power line.
* Why did they accidentally cut the power

In [105]:
search_results = web_search_tool.invoke({"query":"What is 5 whys techniques? Can you explain it to me with one or two examples?"})

In [108]:
search_results[0].keys()

dict_keys(['url', 'content'])

In [110]:
search_results[0]['content']

"Here are six steps to implementing Toyoda's five whys method: 1. Assemble a team. Choose a team that can give you insights into the inner workings of the relevant department or area. You should include management and employees. The best responses come from employees who have first-hand knowledge of the situation."

In [111]:
from langchain_community.tools import DuckDuckGoSearchRun

In [113]:
from langchain_community.document_loaders import WebBaseLoader

In [118]:
urls = [url['url'] for url in search_results]

In [128]:
urls

['https://www.indeed.com/career-advice/career-development/5-whys-example',
 'https://reliability.com/resources/articles/5-whys-root-cause-analysis/',
 'https://buffer.com/resources/5-whys-process/',
 'https://www.masterclass.com/articles/how-to-use-the-5-whys-technique-for-a-root-cause-analysis',
 'https://www.mindtools.com/a3mi00v/5-whys']

In [130]:
search_results

[{'url': 'https://www.indeed.com/career-advice/career-development/5-whys-example',
  'content': "Here are six steps to implementing Toyoda's five whys method: 1. Assemble a team. Choose a team that can give you insights into the inner workings of the relevant department or area. You should include management and employees. The best responses come from employees who have first-hand knowledge of the situation."},
 {'url': 'https://reliability.com/resources/articles/5-whys-root-cause-analysis/',
  'content': "Step 1: Identify the Problem. Before diving into a 5 Whys analysis, it's crucial to clearly identify the problem or issue at hand. This step sets the stage for the entire process and ensures that the focus remains on addressing the right concern. Take the time to gather relevant data, observe patterns, and consult with team members or ..."},
 {'url': 'https://buffer.com/resources/5-whys-process/',
  'content': 'Step 2: Select a 5 Whys master for the meeting. The 5 Whys master will le

In [131]:
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

In [132]:
len(docs)

5

In [133]:
len(docs[0])

1

In [134]:
docs[3]

[Document(page_content='Just a moment...Enable JavaScript and cookies to continue', metadata={'source': 'https://www.masterclass.com/articles/how-to-use-the-5-whys-technique-for-a-root-cause-analysis', 'title': 'Just a moment...', 'language': 'en-US'})]

In [122]:
print(docs[0])

[Document(page_content='\n\n\n\n  \n\nAttention Required! | Cloudflare\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPlease enable cookies.\n\n\nSorry, you have been blocked\nYou are unable to access indeed.com\n\n\n\n\n\n\n\n\n\n\n\nWhy have I been blocked?\nThis website is using a security service to protect itself from online attacks. The action you just performed triggered the security solution. There are several actions that could trigger this block including submitting a certain word or phrase, a SQL command or malformed data.\n\n\nWhat can I do to resolve this?\nYou can email the site owner to let them know you were blocked. Please include what you were doing when this page came up and the Cloudflare Ray ID found at the bottom of this page.\n\n\n\n\n\nCloudflare Ray ID: 88a7f5b15c4d2721\n•\n\n      Your IP:\n      Click to reveal\n184.22.35.227\n•\n\nPerformance & security by Cloudflare\n\n\n\n\n\n\n\n\n', metadata={'source': 'https://www.indeed.com/career-advice/career-development/5-whys-exampl

In [135]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [137]:
@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

print(search.name)
print(search.description)
print(search.args)

search
Look up things online.
{'query': {'title': 'Query', 'type': 'string'}}


In [140]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [142]:
llm_with_tools = agent_llm.bind_tools([multiply])

In [147]:
llm_with_tools = agent_llm.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

In [149]:
from langchain_core.messages import HumanMessage

llm_with_tools.invoke("what is the weather in Boston?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "Boston, MA", "unit": "fahrenheit"}'}}, id='run-e2445e36-b3e2-469c-a152-34337334e65f-0')